[reference](https://github.com/udacity/deep-reinforcement-learning/blob/master/reinforce/REINFORCE.ipynb) <br>
[pytorch ](https://pytorch.org/docs/stable/distributions.html) <br>
[1st rank cartpole](https://github.com/udacity/deep-reinforcement-learning/blob/master/hill-climbing/Hill_Climbing.ipynb)

In [1]:
%reload_ext autoreload
%autoreload 2
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv
import torch
import random
import numpy as np
import math
from EXITrl.table_base import TableBase
from EXITrl.approx_v_base import ApproxVBase
from EXITrl.approx_policy_base import ApproxPolicyBase
from gridworld_env_2d_state import GridworldEnv2DState
import gym

In [2]:
try: env.close()
except: pass
env = gym.make('CartPole-v1')

class REINFORCE(ApproxPolicyBase):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.initialize()
    
    def _loop(self, episode) -> int:
        done = False
        total_reward, reward = 0, 0
        state = self.env.reset()
        trajectory = []
        for i in range(1000):
            action, log_prob = self.policy(state)
            _state, reward, done, _ = self.env.step(action)
            trajectory.append((state, reward, done, log_prob))
            total_reward += reward
            state = _state
            if done: break
                
        loss = []
        for t in range(len(trajectory)):
            G = sum([self.gamma**k * reward for k, (state, reward, done, _) in enumerate(trajectory[t:])])
            _, _, _, log_prob = trajectory[t]
            # from Sutton's book said multiply by gamma**t 
            # but from David course no need too 
            # I tried both work
            loss.append(self.gamma**t * G * (-log_prob))
        self.update_policy(torch.stack(loss).sum())
        return total_reward
    
s = REINFORCE(env, 
               num_episodes=500,
               policy="softmax_policy",
               alpha=0.007, 
               gamma=.99)
s.train(True)


/Users/epinyoanun/miniconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Episode 10	Average Score: 20.80 	other{}
Episode 20	Average Score: 19.00 	other{}
Episode 30	Average Score: 34.20 	other{}
Episode 40	Average Score: 26.80 	other{}
Episode 50	Average Score: 24.90 	other{}
Episode 60	Average Score: 32.90 	other{}
Episode 70	Average Score: 39.30 	other{}
Episode 80	Average Score: 58.50 	other{}
Episode 90	Average Score: 60.10 	other{}
Episode 100	Average Score: 45.70 	other{}
Episode 110	Average Score: 54.40 	other{}
Episode 120	Average Score: 51.70 	other{}
Episode 130	Average Score: 56.50 	other{}
Episode 140	Average Score: 61.40 	other{}
Episode 150	Average Score: 71.90 	other{}
Episode 160	Average Score: 70.50 	other{}
Episode 170	Average Score: 122.10 	other{}
Episode 180	Average Score: 136.10 	other{}
Episode 190	Average Score: 164.10 	other{}
Episode 200	Average Score: 134.10 	other{}
Episode 210	Average Score: 179.30 	other{}
Episode 220	Average Sco

In [6]:
env.close()

### REINFORCE with Baseline

In [6]:
try: env.close()
except: pass
env = gym.make('CartPole-v1')

class REINFORCE_with_baseline(ApproxVBase, ApproxPolicyBase):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        hidden = 64
        features = torch.nn.Linear(self.num_state, hidden)
        model_q = torch.nn.Sequential(
            features,
            torch.nn.ReLU(),
            torch.nn.Linear(hidden, hidden),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden, self.num_action),
        ).to(self.device)
        model_policy = torch.nn.Sequential(
            features,
            torch.nn.ReLU(),
            torch.nn.Linear(hidden, hidden),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden, self.num_action),
        ).to(self.device)
        ApproxVBase.initialize(self, learning_rate_name="beta", model=model_q)
        ApproxPolicyBase.initialize(self, learning_rate_name="alpha", model=model_policy)
    
    def _loop(self, episode) -> int:
        done = False
        total_reward, reward = 0, 0
        state = self.env.reset()
        trajectory = []
        for i in range(1000):
            action, log_prob = self.policy(state)
            _state, reward, done, _ = self.env.step(action)
            trajectory.append((state, action, reward, done, log_prob))
            total_reward += reward!
            state = _state
            if done: break
                
        td_targets = []
        qs = []
        loss_policy = []
        for t in range(len(trajectory)):
            G = sum([self.gamma**k * reward for k, (state, action, reward, done, log_prob) in enumerate(trajectory[t:])])
            state, action, reward, done, log_prob = trajectory[t]
            td_error = G - self.get_q(state)[action]
            td_targets.append(G)
            qs.append(self.get_q(state)[action])
            loss_policy.append(self.gamma**t * td_error * (-log_prob))
        self.update_q(torch.Tensor(td_targets), torch.stack(qs))
        self.update_policy(torch.stack(loss_policy).sum())
        return total_reward
    
s = REINFORCE_with_baseline(env, 
               num_episodes=500,
               policy="softmax_policy",
               alpha=0.007, 
               beta=0.001,
               gamma=.99)
s.train(True)


/Users/epinyoanun/miniconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Episode 10	Average Score: 18.00 	other{}
Episode 20	Average Score: 10.80 	other{}
Episode 30	Average Score: 9.50 	other{}}
Episode 40	Average Score: 9.60 	other{}
Episode 50	Average Score: 9.20 	other{}
Episode 60	Average Score: 9.10 	other{}
Episode 70	Average Score: 9.20 	other{}
Episode 80	Average Score: 9.40 	other{}
Episode 90	Average Score: 9.50 	other{}
Episode 100	Average Score: 9.20 	other{}
Episode 110	Average Score: 19.20 	other{}
Episode 120	Average Score: 53.40 	other{}
Episode 130	Average Score: 14.00 	other{}
Episode 140	Average Score: 9.60 	other{}}
Episode 150	Average Score: 9.20 	other{}
Episode 160	Average Score: 9.50 	other{}
Episode 170	Average Score: 9.50 	other{}
Episode 180	Average Score: 9.40 	other{}
Episode 190	Average Score: 9.30 	other{}
Episode 200	Average Score: 8.80 	other{}
Episode 210	Average Score: 9.10 	other{}
Episode 220	Average Score: 8.80 	other{}
E

### It's really bad for table base problem

In [4]:
env = GridworldEnv2DState()
s = REINFORCE(env, 
               num_episodes=100,
               policy="softmax_policy",
               epsilon=0.01, 
               alpha=0.008, 
               gamma=.9)
s.train(True)

Episode 10	Average Score: -18.80 	other{}
Episode 20	Average Score: -12.20 	other{}
Episode 30	Average Score: -12.20 	other{}
Episode 40	Average Score: -19.50 	other{}
Episode 50	Average Score: -30.40 	other{}
Episode 60	Average Score: -18.80 	other{}
Episode 70	Average Score: -6.70 	other{}}
Episode 80	Average Score: -11.70 	other{}
Episode 90	Average Score: -10.10 	other{}
Episode 100	Average Score: -9.10 	other{}
